In [0]:
import pyspark

In [0]:
from pyspark.context import SparkContext

In [0]:
sc = SparkContext.getOrCreate()

In [0]:
autoData = sc.textFile("/FileStore/tables/Stores.csv")
autoData.cache()

Out[4]: /FileStore/tables/Stores.csv MapPartitionsRDD[100] at textFile at NativeMethodAccessorImpl.java:0

In [0]:
dataLines = autoData.filter(lambda x: "Store_Sales" not in x)
dataLines.count()

Out[5]: 896

In [0]:
import math
from pyspark.mllib.linalg import Vectors


In [0]:
avgHP =sc.broadcast(80.0)

In [0]:
def transformToNumeric( inputString) :
    global avgHP
    attList=inputStr.split(",")
    
    #Replace ? values with a normal value
    hpValue = attList[3]
    if hpValue == "?":
        hpValue=avgHP.value
       
    #Filter out columns not wanted at this stage
    values= Vectors.dense([ float(attList[0]), \
                     float(attList[1]),  \
                     hpValue,    \
                           
                     float(attList[5]),  \
                     float(attList[6])
                     ])
    return values

In [0]:
stores = dataLines.map(transformToNumeric)
stores.collect()

In [0]:
from pyspark.mllib.stat import Statistics
autoStats=Statistics.colStats(stores)
autoStats.mean()
autoStats.variance()
autoStats.min()
autoStats.max()

In [0]:
Statistics.corr(stores)


Out[12]: array([[ 1.        , -0.77539629, -0.77463084,  0.42028891,  0.57926713],
       [-0.77539629,  1.        ,  0.84275215, -0.50541949, -0.3487458 ],
       [-0.77463084,  0.84275215,  1.        , -0.68829885, -0.41559383],
       [ 0.42028891, -0.50541949, -0.68829885,  1.        ,  0.28813695],
       [ 0.57926713, -0.3487458 , -0.41559383,  0.28813695,  1.        ]])

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

/databricks/spark/python/pyspark/sql/context.py:82: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
def transformToLabeledPoint(inStr) :
    lp = ( float(inStr[0]), Vectors.dense([inStr[1],inStr[2],inStr[4]]))
    return lp

In [0]:
autoLp = autoVectors.map(transformToLabeledPoint)
autoDF = sqlContext.createDataFrame(autoLp,["label", "features"])
autoDF.select("label","features").show(10)

+-----+----------------+
|label|        features|
+-----+----------------+
| 18.0|[8.0,130.0,70.0]|
| 15.0|[8.0,165.0,70.0]|
| 18.0|[8.0,150.0,70.0]|
| 16.0|[8.0,150.0,70.0]|
| 17.0|[8.0,140.0,70.0]|
| 15.0|[8.0,198.0,70.0]|
| 14.0|[8.0,220.0,70.0]|
| 14.0|[8.0,215.0,70.0]|
| 14.0|[8.0,225.0,70.0]|
| 15.0|[8.0,190.0,70.0]|
+-----+----------------+
only showing top 10 rows



In [0]:
#Find correlations
numFeatures = autoDF.take(1)[0].features.size
labelRDD = autoDF.map(lambda lp: float(lp.label))
for i in range(numFeatures):
    featureRDD = autoDF.map(lambda lp: lp.features[i])
    corr = Statistics.corr(labelRDD, featureRDD, 'pearson')
    print('%d\t%g' % (i, corr))

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-4056948263047560> in <module>
      1 #Find correlations
      2 numFeatures = autoDF.take(1)[0].features.size
----> 3 labelRDD = autoDF.map(lambda lp: float(lp.label))
      4 for i in range(numFeatures):
      5     featureRDD = autoDF.map(lambda lp: lp.features[i])

/databricks/spark/python/pyspark/sql/dataframe.py in __getattr__(self, name)
   1798         """
   1799         if name not in self.columns:
-> 1800             raise AttributeError(
   1801                 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
   1802         jc = self._jdf.apply(name)

AttributeError: 'DataFrame' object has no attribute 'map'

In [0]:
#Split into training and testing data
(trainingData, testData) = autoDF.randomSplit([0.9, 0.1])
trainingData.count()
testData.count()


Out[18]: 36

In [0]:
#Build the model on training data
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=10)
lrModel = lr.fit(trainingData)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-4056948263047562> in <module>
      2 from pyspark.ml.regression import LinearRegression
      3 lr = LinearRegression(maxIter=10)
----> 4 lrModel = lr.fit(trainingData)

/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method(self, *args, **kwargs)
     28             call_succeeded = False
     29             try:
---> 30                 result = original_method(self, *args, **kwargs)
     31                 call_succeeded = True
     32                 return result

/databricks/spark/python/pyspark/ml/base.py in fit(self, dataset, params)
    159                 return self.copy(params)._fit(dataset)
    160             else:
--> 161                 return self._fit(dataset)
    162         else:
    163             raise TypeError("Params must be either a param map or a list/tuple of par

In [0]:
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4056948263047563> in <module>
----> 1 print("Coefficients: " + str(lrModel.coefficients))
      2 print("Intercept: " + str(lrModel.intercept))

NameError: name 'lrModel' is not defined

In [0]:
#Predict on the test data
predictions = lrModel.transform(testData)
predictions.select("prediction","label","features").show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4056948263047564> in <module>
      1 #Predict on the test data
----> 2 predictions = lrModel.transform(testData)
      3 predictions.select("prediction","label","features").show()

NameError: name 'lrModel' is not defined

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
evaluator.evaluate(predictions)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4056948263047565> in <module>
      2 evaluator = RegressionEvaluator(predictionCol="prediction", \
      3                  labelCol="label",metricName="r2")
----> 4 evaluator.evaluate(predictions)

NameError: name 'predictions' is not defined